In [ ]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns

from amftrack.util.dbx import upload_folders, load_dbx, download
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import (
    read_saved_dropbox_state,
    get_dropbox_folders_prince,
    get_dropbox_folders_prince,
    save_dropbox_state,
    get_dropbox_folders_general_recursive,
)

In [ ]:
# cut
plates = [
    "449_20230807",
]
directory_targ = "/projects/0/einf914/transport/"

In [ ]:
directory_targ = directory_project
# directory_targ = directory_project+'transport/'
# directory_targ = '/projects/0/einf914/transport/'

In [ ]:
# A5s

plates = [
    "142_20240327",
    "143_20240330",
    "613_20240330",
    "621_20240330",
]
directory_targ = directory_project


In [ ]:
# A5s

plates = [
    # "195_20240219",
    # "183_20240219",
    # "215_20240212",
    # "171_20240212",
    # "123_20240209",
    # "203_20240216",
    # "103_20240228",
    "68_20230501",
    # "135_20240228",
]
plates += [
# "440_20240306",
# "435_20240223",
# "441_20240329",
# "443_20240304",
]
directory_targ = os.path.join(directory_scratch, "stitch_temp") + "/"

In [ ]:
dir_drop = "DATA/PRINCE"
all_folders_drop = get_dropbox_folders_prince("/DATA/PRINCE")
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(folders["folder"])]
folders_drop3 = folders_drop2.loc[~folders_drop2["folder"].str.contains("Analysis")]

In [ ]:
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
# folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = folders.sort_values("datetime")

In [ ]:
all_folders_drop

In [ ]:
import shutil

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = all_folders

for path in folders["total_path"]:
    shutil.rmtree(path)

In [ ]:
# name_job = f"441Tryone_{time_ns()}.sh"
# name_job = f"chicory"
name_job = f"Myr"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, -1, "post_processer_choice.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "cut"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 2, "stitcher.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

In [ ]:
plates

In [ ]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "A5sup2"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30, "post_processer_1.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

In [ ]:
directory_targ

In [ ]:
name_job = "rerealign"
run_launcher(
    "realigner.py",
    [directory_targ, name_job, 20],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
name_job = "Myr"
run_launcher(
    "stitcher_myr.py",
    [directory_targ, name_job, 1],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
plates

In [ ]:
name_job = "upload"
run_launcher(
    "analysis_uploader_no_upload.py",
    [directory_targ, name_job, 1],
    plates,
    "6:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
len(folders)

In [ ]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [ ]:
analysis_folders

In [ ]:
name_job = "agg_clarum"
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 100
time = "6:00:00"
hyph_width = 30
perc_low = 85
perc_high = 99.5
minlow = 10
minhigh = 70

args = [hyph_width, perc_low, perc_high, minlow, minhigh, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
print(len(folders))
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
print(len(folders))

In [ ]:
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders1 = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
    folders2 = folders.loc[folders["/Analysis/skeleton_compressed.mat"] == True]
    print(
        plate,
        len(folders1),
        len(folders2),
        folders1["folder"].sort_values().iloc[0],
        folders2["folder"].sort_values().iloc[0],
    )
    # if plate == '13_20220422':
    #     break

In [ ]:
folders1["folder"].sort_values(), folders2["folder"]

In [ ]:
folders = all_folders

In [ ]:
name_job = "empty"
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    70,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

In [ ]:
plates = [plate for plate in all_folders["unique_id"].unique() if plate not in plates]

In [ ]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

args = [directory_targ]
num_parallel = 32
name_job = "agg_rerun"
time = "6:00:00"
run_parallel_all_time(
    "make_labeled_graphs.py",
    args,
    folders,
    num_parallel,
    time,
    "make_graphs",
    cpus=32,
    node="thin",
    dependency=True,
    name_job=name_job,
)

In [ ]:
all_folders["unique_id"].unique()

In [ ]:
folders.loc[folders["/Analysis/transform.mat"] == True]

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"
stage = 30
args = [directory_targ]
for plate in plates:
    select = folders.loc[folders["unique_id"] == plate]
    num_parallel = 32
    time = "10:00"
    args = [directory_targ]
    run_parallel(
        "realign.py",
        args,
        select,
        num_parallel,
        time,
        "create_realign",
        cpus=32,
        node="fat",
        name_job=name_job,
    )

run_launcher(
    "graph_extracter.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
name_job = "rerun"
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 10],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

In [ ]:
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import run_parallel_post

name_job = "malin"
time = "1:00:00"
list_f = [plot_tracking]
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ
list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 100
run_parallel_post(
    "exp_plot.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_plate_post_process",
    cpus=128,
    name_job=name_job,
    node="fat",
)

In [ ]:
from amftrack.pipeline.functions.post_processing.time_hypha import *
import pandas as pd
import os

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "10:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

In [ ]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
args = [directory_targ]
num_parallel = 32
time = "1:00:00"
limit = 200
version = 1
suffix = "_labeled"
lim_considered = 10
args = [directory_targ, limit, version, suffix, lim_considered]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
    name_job="test",
)

In [ ]:
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
len(folders)

In [ ]:
len(list(set(folders["unique_id"].values)))

In [ ]:
stage = 20
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "20:00",
    dependency=True,
    name_job="test",
)

In [ ]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [ ]:
analysis_folders

In [ ]:
# name_job = f"one_shot_{time_ns()}.sh"

from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "24:00:00"
directory = directory_targ
max_ind = 20
incr = 100
fs = [
    get_density_in_ring,
    get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 30
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "compress_image.py",
    args,
    folders,
    num_parallel,
    time,
    "compress",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 50
time = "12:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "detect_blob.py",
    args,
    folders,
    num_parallel,
    time,
    "detect_blob",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
name_job = "malin"

run_launcher(
    "masker.py",
    [directory_targ, name_job, 30],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 50
time = "10:00"
args = []
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job="video",
)

In [ ]:
plates = [
    "105_20220817",
    "77_20220808",
    "71_20220817",
    "100_20220812",
    "117_20220815",
    "99_20220815",
    "75_20220808",
    "226_20220812",
]

In [ ]:
folders = all_folders.loc[all_folders["Plate"].isin(["32"])]

In [ ]:
from amftrack.pipeline.launching.run_super import *

num_parallel = 32
time = "8:00:00"
limit = 200
version = 1
labeled = True
args = [directory_targ, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

In [ ]:
plates

In [ ]:
select

In [ ]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [ ]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

In [ ]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )